In [1]:
import pandas as pd
import imblearn
import numpy as np
import traceback
pd.set_option('display.max_rows', 20,'max_info_columns', 9999,'display.max_columns', 9999)
import matplotlib.pyplot as plt
import copy
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing,metrics
import datetime as dt
import gc
from sklearn import svm,linear_model
from collections import Counter
from sklearn.model_selection import train_test_split
import csv
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder,Imputer

from sklearn.ensemble import RandomForestRegressor,VotingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn import neighbors
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedKFold


import jieba #导入结巴分词
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU


from sklearn.feature_selection import RFE 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
import xlrd
warnings.filterwarnings('ignore')

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [1]:
print([x*(x+1) for x in range(1,100,2)])

[2, 12, 30, 56, 90, 132, 182, 240, 306, 380, 462, 552, 650, 756, 870, 992, 1122, 1260, 1406, 1560, 1722, 1892, 2070, 2256, 2450, 2652, 2862, 3080, 3306, 3540, 3782, 4032, 4290, 4556, 4830, 5112, 5402, 5700, 6006, 6320, 6642, 6972, 7310, 7656, 8010, 8372, 8742, 9120, 9506, 9900]


In [2]:
train=pd.read_csv('/home/tsl-yu/文档/云移杯/YNU.EDU2018-ScenicWord/train_first.csv')

test=pd.read_csv('/home/tsl-yu/文档/云移杯/YNU.EDU2018-ScenicWord/predict.csv')

In [3]:
train.head()

,Id,Discuss,Score
0,201e8bf2-77a2-3a98-9fcf-4ce03914e712,好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的,5
1,f4d51947-eac4-3005-9d3c-2f32d6068a2d,新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！,4
2,74aa7ae4-03a4-394c-bee0-5702d3a3082a,庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去,4
3,099661c2-4360-3c49-a2fe-8c783764f7db,个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...,5
4,97ca672d-e558-3542-ba7b-ee719bba1bab,迪斯尼一日游,5


In [6]:
set(train['Score'])

{1, 2, 3, 4, 5}

In [4]:
cw = lambda x: list(jieba.cut(x)) #定义分词函数
train['words'] = train['Discuss'].apply(cw)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.826 seconds.
Prefix dict has been built succesfully.


In [18]:
y = np_utils.to_categorical(train['Score']-1)

In [20]:
y.shape

(100000, 5)

In [6]:
predict=test

In [7]:
predict['words'] = predict['Discuss'].apply(cw)

### 统计詞频 

In [8]:
d2v_train = pd.concat([train['words'], predict['words']], ignore_index = True)

In [10]:
#将所有词语整合在一起
w = []
for i in d2v_train:
  w.extend(i)


In [16]:
dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数
#del w,d2v_train

In [17]:
dict['id']=list(range(1,len(dict)+1))
# dict = dict[(dict[0]>=10) & (dict[0] < 72410)]
# print(dict)


In [18]:
dict

,0,id
，,364269,1
的,210455,2
。,123441,3
,72410,4
了,56857,5
是,54971,6
很,39654,7
去,34901,8
在,32626,9
有,28918,10


In [21]:
get_sent = lambda x: list(dict['id'][x])

In [22]:
train['sent'] = train['words'].apply(get_sent) #速度太慢
predict['sent'] = predict['words'].apply(get_sent) #速度太慢

In [24]:
maxlen = 10

In [25]:
print("Pad sequences (samples x time)")
train['sent'] = list(sequence.pad_sequences(train['sent'], maxlen=maxlen))
predict['sent'] = list(sequence.pad_sequences(predict['sent'], maxlen=maxlen))

Pad sequences (samples x time)


In [26]:
train['sent'][2]

array([  205,    82,   294,     2,  1467,  2662, 92063,     1,   139,
         112], dtype=int32)

In [27]:
x = np.array(list(train['sent']))[::2] #训练集
y_ = np.array(y)[::2]

xt = np.array(list(train['sent']))[1::2] #测试集
yt = np.array(y)[1::2]

x_sub = np.array(list(predict['sent'])) #提交集

In [28]:
x

array([[   35, 23043,  3083, ...,     1, 14236,     2],
       [  205,    82,   294, ...,     1,   139,   112],
       [    0,     0,     0, ...,     0,  1647,   548],
       ...,
       [    0,     0,     0, ...,     0,    16,    12],
       [ 2882,   712,     7, ...,     2,    30,     3],
       [  127,  3519,   116, ...,   256,   108,   159]], dtype=int32)

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(dict) + 1, 256))
model.add(LSTM(256)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(x, y_, batch_size=32, nb_epoch=2,validation_data=(xt,yt)) #训练时间为若干个小时

y_test = [np.argmax(i) for i in list(yt)]
v = pd.DataFrame()
v['result'] = list(y_test)
v['true'] = list(model.predict_classes(xt,batch_size=32))
print(v)

In [4]:
a=np.random.rand(2,3)<0.8

In [5]:
a

array([[ True,  True,  True],
       [False,  True,  True]])

In [6]:
b=np.random.rand(2,3)

In [7]:
b

array([[0.17114804, 0.27230914, 0.8840475 ],
       [0.89157913, 0.02448434, 0.81999735]])

In [8]:
a*b

array([[0.17114804, 0.27230914, 0.8840475 ],
       [0.        , 0.02448434, 0.81999735]])

In [10]:
np.multiply(a,b)/0.8

array([[0.21393505, 0.34038642, 1.10505938],
       [0.        , 0.03060543, 1.02499669]])